In [1]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

TRAIN_DIR = 'E:\\Courses\\youtube\\DeepLearning_with_Tensorflow\\CNN\\Dogs vs. Cats Redux\\train'
TEST_DIR = 'E:\\Courses\\youtube\\DeepLearning_with_Tensorflow\\CNN\\Dogs vs. Cats Redux\\test'
IMAGE_SIZE = 50
LR = 1e-3

MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '6conv-basic')

In [2]:
def label_img(img):
    word_label = img.split('.')[-3]
    return [1,0] if word_label=='cat' else [0,1] if word_label=='dog' else None    

In [3]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMAGE_SIZE, IMAGE_SIZE))
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data) 
    np.save('train_data.npy', training_data)
    return training_data    

In [4]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMAGE_SIZE, IMAGE_SIZE))
        testing_data.append([np.array(img), img_num])
    np.save('test_data.npy', testing_data)
    return testing_data

In [5]:
# train_data = create_train_data()
# if there is already training data:
train_data = np.load('train_data.npy')
not_a_tensor = np.array([train_data[0][0]])
tensor = np.array([train_data[0][0]]).reshape(-1,IMAGE_SIZE,IMAGE_SIZE,1) 
print("Shape of not_a_tensor = ", not_a_tensor.shape) 
print("Shape of tensor = ", tensor.shape)

Shape of not_a_tensor =  (1, 50, 50)
Shape of tensor =  (1, 50, 50, 1)


In [6]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf
tf.reset_default_graph()

convnet = input_data(shape=[None, IMAGE_SIZE, IMAGE_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

E:\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [13]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

INFO:tensorflow:Restoring parameters from E:\Courses\youtube\DeepLearning_with_Tensorflow\CNN\Dogs vs. Cats Redux\dogsvscats-0.001-6conv-basic.model
model loaded!


In [8]:
train = train_data[:-500]
test = train_data[-500:]

In [10]:
X = np.array([i[0] for i in train]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
test_y = [i[1] for i in train]

In [14]:
model.fit({'input': X}, {'targets': Y}, n_epoch=5, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 3829  | total loss: 0.30583 | time: 8.308s
| Adam | epoch: 010 | loss: 0.30583 - acc: 0.8549 -- iter: 24448/24500
Training Step: 3830  | total loss: 0.32127 | time: 9.332s
| Adam | epoch: 010 | loss: 0.32127 - acc: 0.8522 | val_loss: 1.87206 - val_acc: 0.4880 -- iter: 24500/24500
--


In [15]:
model.save(MODEL_NAME)

INFO:tensorflow:E:\Courses\youtube\DeepLearning_with_Tensorflow\CNN\Dogs vs. Cats Redux\dogsvscats-0.001-6conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.
